# CIFAR10 + Transfer Learning Tutorial

## Goals

This will be a simple application of transfer learning for CIFAR10. We are using a pretrained ResNet-18 model.

Next, we can finally start working on our project. These are the two papers I found, but you can look for more.

1. [Real-time American Sign Language Recognition with Convolutional Neural
Networks](http://cs231n.stanford.edu/reports/2016/pdfs/214_Report.pdf)
2. [Using Deep Convolutional Networks for
Gesture Recognition in American Sign Language](https://arxiv.org/pdf/1710.06836.pdf)

There are also two datasets that we can work with. Part of the group might want to research setting up a dataloader with this dataset in pytorch.

1. [ASL Finger Spelling Dataset](https://empslocal.ex.ac.uk/people/staff/np331/index.php?section=FingerSpellingDataset)
2. [Massey University Gesture Dataset](https://www.massey.ac.nz/~albarcza/gesture_dataset2012.html)

## Resources

Really good notes if you want to get a better understanding of the topics (http://cs231n.github.io/).

## Import Libraries

In [2]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Miscellaneous Libraries
import time

## Set Hyperparameters

In [12]:
# Number of epochs for training
num_epochs = 1

# Batch Size for training/testing
batch_size = 10

# Learning Rate for optimizer
learning_rate = 0.01

# Dimensions of model (ResNet-18)
dim = 224

## Setup Data Loader

In [15]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),             # Normalize grayscale data
])

In [16]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

Files already downloaded and verified


## Initialize Model/Optimizer

In [8]:
# Initialize pretrained ResNet-18 model
model = torchvision.models.resnet18(pretrained=True)

# Turn of gradient calculation for all layers (We don't train them)
for param in model.parameters():
    param.requires_grad = False

# Create new final fully-connected layer for our problem
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)                                             

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.fc.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();                                                        

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /home/nirmal/.cache/torch/checkpoints/resnet18-5c106cde.pth
100.0%


## Train Model

In [13]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/1, Step: 1/5000, Loss: 2.339088201522827, Accuracy: 50.0%
Epoch: 1/1, Step: 101/5000, Loss: 0.2584473490715027, Accuracy: 90.0%
Epoch: 1/1, Step: 201/5000, Loss: 1.5821659564971924, Accuracy: 70.0%
Epoch: 1/1, Step: 301/5000, Loss: 1.8110291957855225, Accuracy: 60.0%
Epoch: 1/1, Step: 401/5000, Loss: 1.1028445959091187, Accuracy: 60.0%
Epoch: 1/1, Step: 501/5000, Loss: 3.5460333824157715, Accuracy: 70.0%
Epoch: 1/1, Step: 601/5000, Loss: 2.0474369525909424, Accuracy: 60.0%
Epoch: 1/1, Step: 701/5000, Loss: 3.0690550804138184, Accuracy: 50.0%
Epoch: 1/1, Step: 801/5000, Loss: 1.1711140871047974, Accuracy: 70.0%
Epoch: 1/1, Step: 901/5000, Loss: 3.6495842933654785, Accuracy: 50.0%
Epoch: 1/1, Step: 1001/5000, Loss: 2.691936492919922, Accuracy: 70.0%
Epoch: 1/1, Step: 1101/5000, Loss: 3.1493656635284424, Accuracy: 40.0%
Epoch: 1/1, Step: 1201/5000, Loss: 0.8947970271110535, Accuracy: 80.0%
Epoch: 1/1, Step: 1301/5000, Loss: 0.5981310606002808, Accuracy: 90.0%
Epoch: 1/1, Step: 14

## Test Model

In [17]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 27.839999999999996%
